In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../../measure/LEI/results/hangang_total_leisure_score_adjusted.csv")
nonhangang_df = pd.read_csv("../../../measure/LEI/results/gu_leisure_score.csv")

# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [41]:
# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 여가도_조정점수

gu = gu.merge(
    nonhangang_df[["자치구", "여가도_조정점수"]],
    left_on="SIGUNGU_NM",
    right_on="자치구",
    how="left"
).drop(columns=["자치구"])

# 공원명: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)


In [42]:

# 공원명 종합_여가도_점수 merge
parks = parks.merge(
    hangang_df[["공원명", "종합_여가도_점수"]],
    left_on="park_name",
    right_on="공원명",
    how="left"
).drop(columns=["공원명"])

# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["여가도_조정점수"].min(), parks["종합_여가도_점수"].min())
all_max = max(gu["여가도_조정점수"].max(), parks["종합_여가도_점수"].max())

cmap = plt.cm.RdPu
norm = colors.Normalize(vmin=all_min, vmax=all_max)

In [43]:
parks

,park_id,park_name,gu_name,id,geometry,종합_여가도_점수
0,HP11,강서한강공원,강서구,5,"MULTIPOLYGON (((126.80426 37.59206, 126.81101 ...",43.262557
1,HP09,난지한강공원,마포구,6,"MULTIPOLYGON (((126.88433 37.55432, 126.88411 ...",55.221426
2,HP10,망원한강공원,마포구,7,"MULTIPOLYGON (((126.8927 37.55844, 126.8942 37...",39.177469
3,HP08,양화한강공원,영등포구,8,"MULTIPOLYGON (((126.9064 37.53549, 126.9047 37...",46.545222
4,HP07,여의도한강공원,영등포구,9,"MULTIPOLYGON (((126.90823 37.53402, 126.91426 ...",62.864752
5,HP05,반포한강공원,서초구,10,"MULTIPOLYGON (((126.98041 37.50654, 126.98125 ...",49.679001
6,HP04,잠원한강공원,서초구,11,"MULTIPOLYGON (((126.99992 37.51894, 127.00099 ...",56.781498
7,HP04,잠원한강공원,강남구,12,"MULTIPOLYGON (((127.00858 37.5256, 127.01001 3...",56.781498
8,HP06,이촌한강공원,용산구,13,"MULTIPOLYGON (((126.95203 37.53037, 126.95279 ...",43.262557
9,HP03,뚝섬한강공원,광진구,1,"MULTIPOLYGON (((127.0592 37.5337, 127.06671 37...",47.900942


In [44]:
# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["여가도_조정점수"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["여가도_조정점수"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "여가도_조정점수"],
        aliases=["자치구", "최종 여가도 점수"]
    )
).add_to(m)

# 공원명 레이어
folium.GeoJson(
    parks,
    name="공원명",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["종합_여가도_점수"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["종합_여가도_점수"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "종합_여가도_점수"],
        aliases=["공원", "최종 여가도 점수"]
    )
).add_to(m)

In [45]:
# 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("../../result/LEI/lei_score_visualization.html")
print("lei_score_visualization.html 저장 완료")

lei_score_visualization.html 저장 완료
